# Data Loading

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import random
from sklearn.impute import KNNImputer
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import scipy.stats as stats
import multiprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.kernel_approximation import Nystroem



In [2]:
#load data
X = np.load("/Users/jiaweizhang/research/data/X.npy")
Y = np.load("/Users/jiaweizhang/research/data/Y.npy")
Z = np.load("/Users/jiaweizhang/research/data/Z.npy")
M = np.load("/Users/jiaweizhang/research/data/M.npy")
S = np.load("/Users/jiaweizhang/research/data/S.npy")

N = len(X)
display(pd.DataFrame(X))
display(pd.DataFrame(Y))
display(pd.DataFrame(Z))
display(pd.DataFrame(M))
display(pd.DataFrame(S))


,0,1,2,3,4
0,0.981770,-0.251813,-0.759265,-2.174064,0.0
1,-0.221323,-1.151682,0.029161,1.107834,0.0
2,1.744473,0.373165,1.073115,1.713880,1.0
3,0.338039,1.880270,-0.630888,-0.832539,1.0
4,0.747007,-1.175739,0.071529,0.313842,0.0
...,...,...,...,...,...
19995,-1.075212,0.356284,0.188095,0.608306,1.0
19996,1.100295,-0.411935,-0.069896,-0.232181,1.0
19997,1.514623,0.329402,1.327177,1.405978,0.0
19998,0.407197,-1.394913,-0.000789,0.542119,1.0


,0,1,2
0,2.773007,-4.307684,28.546945
1,23.131155,-0.293310,0.276195
2,111.466797,111.979334,641.841480
3,50.234558,24.110527,39.479267
4,21.311598,3.856759,1.343088
...,...,...,...
19995,34.241948,6.935492,8.770458
19996,31.320879,16.715269,17.345786
19997,78.006079,71.204181,312.519088
19998,32.060799,10.197790,1.355323


,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
19995,1.0
19996,1.0
19997,1.0
19998,1.0


,0,1,2
0,0.0,0.0,1.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
19995,0.0,0.0,0.0
19996,0.0,0.0,0.0
19997,0.0,0.0,0.0
19998,0.0,0.0,0.0


,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
19995,199.0
19996,199.0
19997,199.0
19998,199.0


# One shot framework

### Randomly split one dataframe to two datasets
split_df takes a pandas DataFrame as input and randomly splits it into two separate DataFrames with a specified proportion of the data in each split. The function shuffles the indices randomly and splits the DataFrame using the shuffled indices. It returns the two separate DataFrames as output.

In [3]:
def split_df_shuffle(df):
    # Set the proportion of data to be split
    split_proportion = 0.5

    # Set a random seed for reproducibility
    random.seed(23)

    # Get the indices for the split
    indices = df.index.tolist()
    num_rows = len(df)
    split_index = int(num_rows * split_proportion)

    # Shuffle the indices randomly
    random.shuffle(indices)

    # Get the randomly selected rows for each split
    split1_indices = indices[:split_index]
    split2_indices = indices[split_index:]

    # Split the original DataFrame into two separate DataFrames
    df1 = df.loc[split1_indices]
    df2 = df.loc[split2_indices]
    
    return df1,df2

#split based on strata
def split_df(df,index_S):

    # Sort the groups by the number of rows in each group
    sorted_df = df.sort_values(by = index_S, ascending=True)
    
    # Split the sorted groups into two equal-sized sets of 100 strata each
    df_set1 = sorted_df.iloc[:int(N/2),0 : index_S]
    df_set2 = sorted_df.iloc[int(N/2):N, 0 : index_S]

    #set the index of the two sets from zero to 1
    df_set1.index = range(len(df_set1))
    df_set2.index = range(len(df_set2))
    
    # Return the two sets of strata
    return df_set1, df_set2


### T-test for T(Z,Y)
the Wilcoxon rank sum test
$T(\mathbf{Z}, \mathbf{Y})=\sum_{n=1}^{N}Z_{n}\cdot \text{rank}(Y_{n})=\sum_{n=1}^{N}\{Z_{n}\cdot \sum_{n^{\prime}=1}^{N} \mathbf{1}(Y_{n}\geq Y_{n^{\prime}})\}$.

In [ ]:
def T(z,y):

    #the Wilcoxon rank sum test
    n = len(z)
    t = 0
    #O(N^2) version
    """
    for n in range(N):
        rank = sum(1 for n_prime in range(N) if Y[n] >= Y[n_prime])
        T += Z[n] * rank
    """

    #O(N*Log(N)) version
    my_list = []
    for i in range(n):
        my_list.append((z[i],y[i]))
    sorted_list = sorted(my_list, key=lambda x: x[1])

    #Calculate
    for i in range(n):
        t += sorted_list[i][0] * (i + 1)
    
    return t

def getT(G, df):
    
    # Get the imputed data Y and indicator Z
    df_imputed = G.transform(df)
    y = df_imputed[:, Z.shape[1] + X.shape[1]:df_imputed.shape[1]]
    z = df_imputed[:, 0]
    
    z_tiled = np.tile(z, 3)

    # Concatenate the tiled versions of Z together
    new_z = np.concatenate((z_tiled,))
    new_y = y.flatten()

    #the Wilcoxon rank sum test
    t = T(new_z,new_y)

    return t



#### t-test

In [ ]:
def ttest(G, df):
    
    # Get the imputed data Y and indicator Z
    df_imputed = G.transform(df)
    Y_pred = df_imputed[:, Z.shape[1] + X.shape[1]:df_imputed.shape[1]]
    Z_shuffled = df_imputed[:, 0]

    # Get the t-statistics for T(Z,Y)
    treatment = Y_pred[Z_shuffled == 1].flatten()
    control = Y_pred[Z_shuffled == 0].flatten()

    t, p = stats.ttest_ind(treatment, control, equal_var=True)

    return t,p

## One Short Framework 


In [ ]:
def one_shot_test(Z, X, M, Y, S, G1, G2,  L=10000, verbose = False):
    """
    A one-shot framework for testing H_0.

    Args:
    Z: 2D array of observed treatment indicators
    X: 2D array of observed covariates
    M: 2D array of observed missing indicators
    Y: 2D array of observed values for K outcomes
    G1: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    G2: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    L: number of Monte Carlo simulations (default is 10000)

    Returns:
    p1: 1D array of exact p-values for testing Fisher's sharp null in part 1
    p2: 1D array of exact p-values for testing Fisher's sharp null in part 2
    """

    #print train start
    if verbose:
        print("Training start")

    # create data a whole data frame
    Y_masked = np.ma.masked_array(Y, mask=M)
    Y_masked = Y_masked.filled(np.nan)
    df = pd.DataFrame(np.concatenate((Z, X, Y_masked,S), axis=1))
    
    # randomly split the data into two parts
    df1, df2 = split_df(df, X.shape[1] + Y.shape[1] + Z.shape[1])

    # impute the missing values and calculate the observed test statistics in part 1
    G1.fit(df1)
    t1_obs = getT(G1, df1)

    # impute the missing values and calculate the observed test statistics in part 2
    G2.fit(df2)
    t2_obs = getT(G2, df2)

    #print train end
    if verbose:
        print("Training end")

    # simulate data and calculate test statistics
    t1_sim = np.zeros(L)
    t2_sim = np.zeros(L)
    
    for l in range(L):

        # simulate treatment indicators in parts 1 and 2
        df_sim = pd.DataFrame(np.concatenate((X, Y_masked, S), axis=1))
        
        # split the simulated data into two parts
        df1_sim, df2_sim = split_df(df_sim, X.shape[1] + Y.shape[1])

        # simulate treatment indicators in parts 1 and 2
        Z_1 = np.random.binomial(1, 0.5, df1_sim.shape[0]).reshape(-1, 1)
        Z_2 = np.random.binomial(1, 0.5, df2_sim.shape[0]).reshape(-1, 1)
        df1_sim = pd.concat([pd.DataFrame(Z_1), df1_sim], axis=1)
        df2_sim = pd.concat([pd.DataFrame(Z_2), df2_sim], axis=1)
        
    
        # get the test statistics in part 1
        t1_sim[l] = getT(G2, df1_sim)

        # get the test statistics in part 2
        t2_sim[l] = getT(G1, df2_sim)

        # Calculate the completeness percentage
        if l % 100 == 0:
            completeness = l / L * 100
            if verbose:  
                print(f"Task is {completeness:.2f}% complete.")

    # calculate exact p-values for each outcome
    p1 = np.mean(t1_sim >= t1_obs, axis=0)
    p2 = np.mean(t2_sim >= t2_obs, axis=0)
    
    return p1, p2
    



###  Parallel Computing Version

In [ ]:
def worker(args):
    # unpack the arguments
    Z, X, M, S, Y_masked, G1, G2, t1_obs, t2_obs, shape, L = args

    # simulate data and calculate test statistics
    t1_sim = np.zeros(L)
    t2_sim = np.zeros(L)

    for l in range(L):

        # simulate treatment indicators in parts 1 and 2
        df_sim = pd.DataFrame(np.concatenate((X, Y_masked, S), axis=1))
        
        # split the simulated data into two parts
        df1_sim, df2_sim = split_df(df_sim, index_S = X.shape[1] + Y.shape[1])

        # simulate treatment indicators in parts 1 and 2
        Z_1 = np.random.binomial(1, 0.5, df1_sim.shape[0]).reshape(-1, 1)
        Z_2 = np.random.binomial(1, 0.5, df2_sim.shape[0]).reshape(-1, 1)
        df1_sim = pd.concat([pd.DataFrame(Z_1), df1_sim], axis=1)
        df2_sim = pd.concat([pd.DataFrame(Z_2), df2_sim], axis=1)

        # get the test statistics in part 1
        t1_sim[l] = getT(G2, df1_sim)

        # get the test statistics in part 2
        t2_sim[l] = getT(G1, df2_sim)

        # Calculate the completeness percentage
        if l % 100 == 0:
            completeness = l / L * 100  
            print(f"Task is {completeness:.2f}% complete.")

    p1 = np.mean(t1_sim >= t1_obs, axis=0)
    p2 = np.mean(t2_sim >= t2_obs, axis=0)

    return p1, p2

def one_shot_test_parallel(Z, X, M, Y, S, G1, G2, L=10000, n_jobs=multiprocessing.cpu_count()):
    """
    A one-shot framework for testing H_0.

    Args:
    Z: 2D array of observed treatment indicators
    X: 2D array of observed covariates
    M: 2D array of observed missing indicators
    Y: 2D array of observed values for K outcomes
    G1: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    G2: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    L: number of Monte Carlo simulations (default is 10000)

    Returns:
    p1: 1D array of exact p-values for testing Fisher's sharp null in part 1
    p2: 1D array of exact p-values for testing Fisher's sharp null in part 2
    """
    #print train start
    print("Training start")

    # create data a whole data frame
    Y_masked = np.ma.masked_array(Y, mask=M)
    Y_masked = Y_masked.filled(np.nan)
    df = pd.DataFrame(np.concatenate((Z, X, Y_masked, S), axis=1))
    
    # randomly split the data into two parts
    df1, df2 = split_df(df, index_S = Z.shape[1] + X.shape[1] + Y.shape[1])

    # impute the missing values and calculate the observed test statistics in part 1
    G1.fit(df1)
    t1_obs = getT(G1, df1)

    # impute the miassing values and calculate the observed test statistics in part 2
    G2.fit(df2)
    t2_obs = getT(G2, df2)

    #print train end
    print("Training end")
    
    # print the number of cores
    print(f"Number of cores: {n_jobs}")


    # simulate data and calculate test statistics in parallel
    args_list = [(Z, X, M, Y_masked, S, G1, G2, t1_obs, t2_obs, df.shape, int(L / n_jobs))] * n_jobs
    with multiprocessing.Pool(processes=n_jobs) as pool:
        p_list = pool.map(worker, args_list)
    p1 = np.mean([p[0] for p in p_list], axis=0)
    p2 = np.mean([p[1] for p in p_list], axis=0)
    
    return p1, p2


# Test the framework 

Test all the machine learning method in single core

### (i)  MissForest
missForest is an algorithm for data imputation, which is the process of filling in missing values in a dataset. missForest is popular, and turns out to be a particular instance of different sequential imputation algorithms that can all be implemented with IterativeImputer by passing in different regressors to be used for predicting missing feature values. In the case of missForest, this regressor is a Random Forest. See Imputing missing values with variants of IterativeImputer.

missForest is an implementation of the random forest algorithm for missing data imputation. The algorithm works by building an ensemble of decision trees to predict the missing values in a dataset. The idea behind the algorithm is that decision trees can be used to model the relationship between the variables in a dataset and can be used to predict missing values. The algorithm works by splitting the dataset into several smaller datasets, building decision trees on each of these smaller datasets, and combining the predictions from these decision trees to obtain a final imputed dataset.

One of the advantages of using missForest is that it can handle missing values in both categorical and continuous variables. It also handles data with different missing patterns and can be used to impute multiple imputations at once. Additionally, missForest provides a measure of the imputation uncertainty, which is important for correctly interpreting the results of the imputed data. 

source - https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py

In [ ]:
#MissForest
print("One-shot test for Fisher's sharp null using MissForest")
missForest = IterativeImputer(estimator = RandomForestRegressor(),max_iter=10, random_state=0)

p1, p2 = one_shot_test(Z, X, M, Y, S,G1=missForest, G2=missForest)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

### (ii) KNN Imputation
The basic idea behind KNN for imputation is to replace missing values with the average of the k-nearest neighbors in the feature space. The value of k is determined by the user and can be set using cross-validation. KNN imputation is considered a simple and effective method for imputing missing data, particularly for small amounts of missing values. However, for larger amounts of missing data or for data with a large number of features, more advanced imputation methods may be needed.

Source - https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer


In [ ]:
#KNN
print("One-shot test for Fisher's sharp null using KNN")
KNNimputer = KNNImputer(n_neighbors=7)
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=KNNimputer, G2=KNNimputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)


### (iii) BayesianRidge Imputation
The BayesianRidge model tries to estimate the coefficients of a linear regression model that best fit the data, taking into account prior knowledge about the coefficients. For data imputation, the missing values are treated as if they are unknown coefficients and are estimated along with the other coefficients during the model fitting process. BayesianRidge can be a good choice for imputation when the relationship between the features is well approximated by a linear model. However, it may not perform well for data sets with more complex relationships between the features.

Source - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html#sklearn.linear_model.BayesianRidge

In [ ]:
#BayesianRidge
print("One-shot test for Fisher's sharp null")
BayesianRidge = IterativeImputer(estimator = linear_model.BayesianRidge(),max_iter=10, random_state=0)
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=BayesianRidge, G2=BayesianRidge)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

### (iv) Nystroem Method for Kernel Approximation
The Nystroem method, as implemented in Nystroem is a general method for low-rank approximations of kernels. It achieves this by essentially subsampling the data on which the kernel is evaluated. By default Nystroem uses the rbf kernel, but it can use any kernel function or a precomputed kernel matrix. The number of samples used - which is also the dimensionality of the features computed - is given by the parameter n_components.

Source - https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html#sklearn.kernel_approximation.Nystroem

In [ ]:
#Nystroem Method for Kernel Approximation
print("One-shot test for Fisher's sharp null using Nystroem Method for Kernel Approximation")
pipeline = make_pipeline(
    StandardScaler(),
    Nystroem(), 
    linear_model.Ridge()
)
NystroemKernel = IterativeImputer(estimator = pipeline,max_iter=10, random_state=0)
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=NystroemKernel, G2=NystroemKernel)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

### (V) XGBoost
XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. The same code runs on major distributed environment (Hadoop, SGE, MPI) and can solve problems beyond billions of examples.


source - https://xgboost.readthedocs.io/en/stable/index.html

In [ ]:
#XGBoost
print("One-shot test for Fisher's sharp null using XGBoost")
pipeline = make_pipeline(
    StandardScaler(),
    xgb.XGBRegressor()
)
XGBoost = IterativeImputer(estimator = pipeline,max_iter=10, random_state=0)
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=XGBoost, G2=XGBoost)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

### (vi) MLPRegressor

MLPRegressor is a class in the scikit-learn library that implements a multi-layer perceptron (MLP) that trains using backpropagation with no activation function in the output layer, which can also be seen as using the identity function as activation function. It is an artificial neural network model that uses backpropagation to adjust the weights between neurons in order to improve prediction accuracy. MLPRegressor trains iteratively since at each time step the partial derivatives of the loss function with respect to the model parameters are computed to update the parameters. It can also have a regularization term added to the loss function that shrinks model parameters to prevent overfitting



In [ ]:
#Neural Network
print("One-shot test for Fisher's sharp null using Neural Network")
pipeline = make_pipeline(
    StandardScaler(),
    MLPRegressor(hidden_layer_sizes=(100, 100, 100), activation='relu', alpha=0.0001, random_state=0)
)

NN_imputer = IterativeImputer(estimator=pipeline.named_steps['mlpregressor'], max_iter=10, random_state=0)

# Assuming the one_shot_test() function is already defined
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=NN_imputer, G2=NN_imputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

### (vii) Median and Mean Imputer for data imputation

Mean and Median as imputed value

source - https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

In [ ]:
#Median imputer
print("One-shot test for Fisher's sharp null using Median imputer")
median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=median_imputer, G2=median_imputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

In [ ]:
#Mean imputer
print("One-shot test for Fisher's sharp null using Mean imputer")
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=mean_imputer, G2=mean_imputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)
